In [1]:
%load_ext line_profiler
from go_mcts import Go_MCTS
from mcts import MCTS
from bokePolicy import PolicyNet
import torch

In [2]:
pi = PolicyNet()
checkpt = torch.load("v0.5/RL_policy_3.pt", map_location = torch.device("cpu"))
pi.load_state_dict(checkpt["model_state_dict"])
tree = MCTS()
board = Go_MCTS(policy=pi)

In [3]:
%lprun -f tree.do_rollout tree.do_rollout(board, 100)

Timer unit: 1e-07 s

Total time: 395.027 s
File: C:\Users\kyles\Documents\GitHub\BokeGo\policy_net_py\mcts.py
Function: do_rollout at line 44

Line #      Hits         Time  Per Hit   % Time  Line Contents
    44                                               def do_rollout(self, node, n=1):
    45                                                   "Train for n iterations"
    46       101       1210.0     12.0      0.0          for i in range(n):
    47                                                       # Get path to leaf of current search tree
    48       100 2960442500.0 29604425.0     74.9              path = self._descend(node, i + 1)
    49       100       3207.0     32.1      0.0              leaf = path[-1]
    50                                                       # Get result of rollout starting from leaf
    51       100  989780063.0 9897800.6     25.1              score = self._simulate(leaf)
    52       100      40080.0    400.8      0.0              self._backpropaga

In [4]:
%lprun -f tree._simulate tree._simulate(board)

Timer unit: 1e-07 s

Total time: 1.00367 s
File: C:\Users\kyles\Documents\GitHub\BokeGo\policy_net_py\mcts.py
Function: _simulate at line 75

Line #      Hits         Time  Per Hit   % Time  Line Contents
    75                                               def _simulate(self, node):
    76                                                   "Returns the reward for a random simulation (to completion) of `node`"
    77         1         25.0     25.0      0.0          invert_reward = not node.color
    78                                                   while True:
    79        72       2794.0     38.8      0.0              if node.is_terminal():
    80         1       3012.0   3012.0      0.0                  reward = node.reward()
    81         1         21.0     21.0      0.0                  reward = invert_reward^reward
    82         1          4.0      4.0      0.0                  return reward
    83        71   10030864.0 141279.8     99.9              node = node.find_random

In [5]:
%timeit tree._simulate(board)

589 ms ± 18.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%lprun -f board.find_random_child board.find_random_child()

Timer unit: 1e-07 s

Total time: 0.0114244 s
File: C:\Users\kyles\Documents\GitHub\BokeGo\policy_net_py\go_mcts.py
Function: find_random_child at line 53

Line #      Hits         Time  Per Hit   % Time  Line Contents
    53                                               def find_random_child(self):
    54                                                   '''Draws legal move from distribution given by policy. If no
    55                                                   policy is given, a legal move is drawn uniformly.
    56                                                   Returns a copy of the board (Go_MCTS object) after the move has
    57                                                   been played.'''
    58         1         19.0     19.0      0.0          if self.terminal:
    59                                                       return self # Game is over; no moves can be made
    60         1     114225.0 114225.0    100.0          return self.make_move(self.get_move())

In [7]:
%lprun -f tree._descend tree._descend(board, 100)

Timer unit: 1e-07 s

Total time: 6.92987 s
File: C:\Users\kyles\Documents\GitHub\BokeGo\policy_net_py\mcts.py
Function: _descend at line 54

Line #      Hits         Time  Per Hit   % Time  Line Contents
    54                                               def _descend(self, node, total_visits):
    55                                                   "Return a path from root down to leaf via PUCT selection"
    56                                                   # Start at root (current position)
    57         1         27.0     27.0      0.0          path = [node]
    58                                                   while True:
    59                                                       # Is node a leaf?
    60         9        478.0     53.1      0.0              if node not in self.children or not self.children[node]:
    61                                                           # Heuristic: if node is "promising" (i.e. large # of visits),
    62                          

In [8]:
%timeit tree._descend(board, 100)

4.39 s ± 121 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%lprun -f tree._puct_select tree._puct_select(board, 100)

Timer unit: 1e-07 s

Total time: 0.801725 s
File: C:\Users\kyles\Documents\GitHub\BokeGo\policy_net_py\mcts.py
Function: _puct_select at line 92

Line #      Hits         Time  Per Hit   % Time  Line Contents
    92                                               def _puct_select(self, node, total_visits):
    93                                                   "Select a child of node with PUCT"
    94                                           
    95                                                   # Predictor + UCT (PUCT) variant used in AlphaGo
    96         1         32.0     32.0      0.0          def puct(n):
    97                                                       avg_reward = 0 if self.N[n] == 0 else self.Q[n] / self.N[n]
    98                                                       return avg_reward + (self.exploration_weight
    99                                                               * n.get_move_prob(n.last_move)
   100                                           

In [10]:
%timeit tree._puct_select(board, 100)

533 ms ± 14.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%timeit board.get_move_prob(board.last_move)

531 ns ± 28.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
